In [65]:
import numpy as np
import pandas as pd
import yaml
import torch
from copy import deepcopy
from tqdm import tqdm
import matplotlib.pyplot as plt
from celldreamer.eval.eval_utils import normalize_and_compute_metrics
import scipy.sparse as sparse
import scipy as sp

from torch import nn
import scanpy as sc    

from celldreamer.data.scrnaseq_loader import RNAseqLoader
from celldreamer.models.featurizers.category_featurizer import CategoricalFeaturizer
from celldreamer.models.fm.fm import FM
from celldreamer.eval.optimal_transport import wasserstein
import random
from celldreamer.models.base.encoder_model import EncoderModel
from celldreamer.models.base.utils import unsqueeze_right

from celldreamer.paths import DATA_DIR

device  = "cuda" if torch.cuda.is_available() else "cpu"

## Step 1: Initialize data

In [66]:
dataset_config = {'dataset_path': DATA_DIR / 'processed' / 'classifier_experiment_pbmc' / 'pbmc_covid_train.h5ad',
                    'layer_key': 'X_counts',
                    'covariate_keys': ['cell_type'],
                    'conditioning_covariate': 'cell_type',
                    'subsample_frac': 1,
                    'encoder_type': 'learnt_autoencoder',
                    'target_max': 1,
                    'target_min': -1,
                    'one_hot_encode_features': False,
                    'split_rates': [0.90, 0.05, 0.05],
                    'cov_embedding_dimensions': 256}


data_path = dataset_config["dataset_path"]

dataset = RNAseqLoader(data_path=data_path,
                                layer_key=dataset_config["layer_key"],
                                covariate_keys=dataset_config["covariate_keys"],
                                subsample_frac=dataset_config["subsample_frac"], 
                                encoder_type=dataset_config["encoder_type"],)

dataloader = torch.utils.data.DataLoader(dataset,
                                            batch_size=256,
                                            shuffle=True,
                                            num_workers=4, 
                                            drop_last=True)

in_dim = dataset.X.shape[1]
size_factor_statistics = {"mean": dataset.log_size_factor_mu, 
                                  "sd": dataset.log_size_factor_sd}
n_cat = len(dataset.id2cov["cell_type"])

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


KeyboardInterrupt: 

## Step 2: Initialize encoder

In [ ]:
encoder_config = {
    "x0_from_x_kwargs": {
        "dims": [512, 256, 50],
        "batch_norm": True,
        "dropout": False,
        "dropout_p": 0.0
    },
    "learning_rate": 0.001,
    "weight_decay": 0.00001,
    "covariate_specific_theta": False
}

state_dict_path = "/home/icb/alessandro.palma/environment/cfgen/project_folder/experiments/train_autoencoder_pbmc_covid/21d30ca0-932d-4891-9796-938e9033b267/checkpoints/last.ckpt"

In [ ]:
encoder_model = EncoderModel(in_dim=in_dim,
                              n_cat=n_cat,
                              conditioning_covariate=dataset_config["conditioning_covariate"], 
                              encoder_type=dataset_config["encoder_type"],
                              **encoder_config)

encoder_model.load_state_dict(torch.load(state_dict_path)["state_dict"])

encoder_model.eval()

## Initialize FM model

In [ ]:
generative_model_config = {'learning_rate': 0.0001,
                            'weight_decay': 0.00001,
                            'antithetic_time_sampling': True,
                            'sigma': 0.0001
                        }

In [ ]:
ckpt = torch.load("/home/icb/alessandro.palma/environment/cfgen/project_folder/experiments/fm_resnet_autoencoder_pbmc_covid_whole_genome/a075cbb3-f9d8-4c34-bc3d-139ba928ba56/checkpoints/last.ckpt")

denoising_model = ckpt["hyper_parameters"]["denoising_model"]
denoising_model.multimodal = False

In [ ]:
denoising_model.embed_size_factor = True

In [ ]:
print(ckpt["hyper_parameters"]["feature_embeddings"]["cell_type"].embeddings.weight)
feature_embeddings = ckpt["hyper_parameters"]["feature_embeddings"]

Initializations

In [ ]:
generative_model = FM(
            encoder_model=encoder_model,
            denoising_model=denoising_model,
            feature_embeddings=feature_embeddings,
            plotting_folder=None,
            in_dim=512,
            size_factor_statistics=size_factor_statistics,
            encoder_type=dataset_config["encoder_type"],
            conditioning_covariate=dataset_config["conditioning_covariate"],
            model_type=denoising_model.model_type, 
            **generative_model_config  # model_kwargs should contain the rest of the arguments
            )

generative_model.load_state_dict(ckpt["state_dict"])
generative_model.to("cuda")

## Read original data 

In [ ]:
adata_original = sc.read_h5ad(data_path)
adata_original.obs["size_factor"]=adata_original.X.A.sum(1)
X = torch.tensor(adata_original.layers["X_counts"].todense())

In [ ]:
adata_original.obs.cell_type

## Generate and save cells

In [ ]:
unique_classes = np.unique(adata_original.obs.cell_type, return_counts=True)
class_freq_dict = dict(zip(unique_classes[0], unique_classes[1]))

In [51]:
under_5000 = []
under_5000_freq = {}

for ct in class_freq_dict:
    if class_freq_dict[ct] < 5000:
        under_5000 += [ct] * (5000-class_freq_dict[ct])
        under_5000_freq[ct] = (5000-class_freq_dict[ct])
        # under_5000 += [ct] * (class_freq_dict[ct]*5)
        # under_5000_freq[ct] = (class_freq_dict[ct]*5)

## General conditional

In [52]:
size_factor = []
for ct in np.unique(under_5000):
    adata_original_c = adata_original[adata_original.obs["cell_type"]==ct]
    i = np.random.choice(len(adata_original_c), under_5000_freq[ct], replace=True)
    size_factor += list(adata_original_c[i].layers["X_counts"].A.sum(1))

size_factor = torch.log(torch.tensor(size_factor)).unsqueeze(1).cuda()

In [53]:
len(under_5000)

58596

In [54]:
size_factor = size_factor[:58000]
under_5000 = under_5000[:58000]

In [55]:
condition_val = torch.tensor([dataset.id2cov["cell_type"][condition_name] for condition_name in under_5000]).long()

X_generated = generative_model.batched_sample(batch_size=58,
                                            repetitions=1000,
                                            n_sample_steps=2, 
                                            covariate="cell_type", 
                                            covariate_indices=condition_val, 
                                            log_size_factor=size_factor)

X_generated_list= X_generated.to("cpu")

## Merge and plot the generated and real data

In [56]:
dataset_type = ["Real" for _ in range(X.shape[0])] + ["Generated" for _ in range(X_generated.shape[0])]
dataset_type = pd.DataFrame(dataset_type)
dataset_type.columns = ["dataset_type"]
dataset_type["cell_type"] = list(adata_original.obs.cell_type)+under_5000

In [57]:
adata_merged = sc.AnnData(X=torch.cat([X, X_generated], dim=0).numpy(), 
                             obs=dataset_type)

/home/icb/alessandro.palma/miniconda3/envs/celldreamer/lib/python3.10/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [58]:
adata_merged.layers["X_counts"] = adata_merged.X.copy()
adata_merged.var = adata_original.var.copy()

In [59]:
sc.pp.log1p(adata_merged)
sc.tl.pca(adata_merged)
# sc.pp.neighbors(adata_merged)
# sc.tl.umap(adata_merged)

In [60]:
# sc.pl.pca(adata_merged, color="dataset_type",  annotate_var_explained=True)
# sc.pl.umap(adata_merged, color="dataset_type")
# sc.pl.umap(adata_merged, color="cell_type")

In [61]:
# sc.write("/home/icb/alessandro.palma/environment/celldreamer/project_folder/datasets/processed/classifier_experiment/augmented/pbmc_covid_augmented_prop.h5ad", adata_merged)

In [62]:
adata_merged.write_h5ad("/home/icb/alessandro.palma/environment/cfgen/project_folder/datasets/processed/classifier_experiment_pbmc/augmented/rebuttals_augmentations/RARE_pbmc_covid_train_augmented_prop_5000.h5ad")

In [63]:
adata_merged.obs["cell_type"].value_counts()

cell_type
naive thymus-derived CD4-positive, alpha-beta T cell                          66988
natural killer cell                                                           43554
classical monocyte                                                            43154
naive B cell                                                                  37844
CD8-positive, alpha-beta cytotoxic T cell                                     32672
CD4-positive helper T cell                                                    30813
naive thymus-derived CD8-positive, alpha-beta T cell                          27166
central memory CD8-positive, alpha-beta T cell                                10822
non-classical monocyte                                                         7928
regulatory T cell                                                              6895
effector memory CD8-positive, alpha-beta T cell, terminally differentiated     5580
CD16-negative, CD56-bright natural killer cell, human             

In [64]:
ad_5000 = sc.read_h5ad("/home/icb/alessandro.palma/environment/cfgen/project_folder/datasets/processed/classifier_experiment_pbmc/augmented/rebuttals_augmentations/RARE_pbmc_covid_train_augmented_prop_5000.h5ad")